Ноутбук запускался в колабе

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using Tesla T4 GPUs


In [2]:
! pip install datasets transformers seqeval

     |████████████████████████████████| 225kB 28.3MB/s 
     |████████████████████████████████| 2.3MB 43.3MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 245kB 36.1MB/s 
     |████████████████████████████████| 112kB 57.7MB/s 
     |████████████████████████████████| 901kB 52.6MB/s 
     |████████████████████████████████| 3.3MB 51.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=5304cbec8508f5a2fdbd1e3d408ef904dbabd33e02fbf9bfe3c3bfd1ff900546
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [3]:
task = "ner"
batch_size = 8

# Data - RFQ


In [4]:
!gdown --id 1L6dd0FnYqgn-eoQ-gFnBiNji7R1ul9n_

Downloading...
From: https://drive.google.com/uc?id=1L6dd0FnYqgn-eoQ-gFnBiNji7R1ul9n_
To: /content/NER_RFQ_agg.csv
4.39MB [00:00, 23.6MB/s]


In [5]:
!gdown --id 1-5mE9XjocmyCKGlkpW1YGuCnNsGwQioD

Downloading...
From: https://drive.google.com/uc?id=1-5mE9XjocmyCKGlkpW1YGuCnNsGwQioD
To: /content/NER_RFQ_agg_train.csv
3.48MB [00:00, 30.6MB/s]


In [6]:
!gdown --id 1d4er4I7x4VIwy7BsWFpsPuC2z6aZ3P6s

Downloading...
From: https://drive.google.com/uc?id=1d4er4I7x4VIwy7BsWFpsPuC2z6aZ3P6s
To: /content/NER_RFQ_agg_test.csv
100% 909k/909k [00:00<00:00, 14.3MB/s]


In [8]:
import pandas as pd
data = pd.read_csv('NER_RFQ_agg.csv', converters={'tokens': eval, 'ner_tags': eval})

In [9]:
all_tags = set()
for _, row in data.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))

In [10]:
tag_to_idx = {t: i for i, t in enumerate(all_tags)}

In [11]:
label_list = {i: t for i, t in enumerate(all_tags)}

In [12]:
train = pd.read_csv('NER_RFQ_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_RFQ_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

In [13]:
train.ner_tags = train.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])
test.ner_tags = test.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])

In [14]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [15]:
from datasets import load_dataset, load_metric

In [16]:
datasets = load_dataset('pandas', data_files={'train' : 'train.pkl', 'test' : 'test.pkl'})

Using custom data configuration default-663904166726946e


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-663904166726946e/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [17]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 953
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 239
    })
})

# Tokenizer

In [18]:
from transformers import AutoTokenizer
model_checkpoint = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [19]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [20]:
label_all_tokens = False

In [21]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Warning : есть последовательности больше, чем максимально возможная в BERT - 512

In [23]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count     953.000000
mean      727.997901
std       655.440163
min       144.000000
25%       384.000000
50%       557.000000
75%       850.000000
max      7082.000000
dtype: float64

In [24]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count     239.000000
mean      748.744770
std       618.354517
min       202.000000
25%       385.000000
50%       568.000000
75%       898.500000
max      6996.000000
dtype: float64

In [25]:
train_tokenized = tokenized_datasets["train"].to_pandas()
test_tokenized = tokenized_datasets["test"].to_pandas()

In [26]:
train_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",123783R.msg,"[101, 11836, 95269, 51136, 267, 132, 256, 8461...","[-100, 2, 2, -100, -100, -100, -100, -100, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[name, 123783r.msg, yuliya, a., kondratova, <y..."


In [27]:
test_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",123670R.msg,"[101, 11836, 97415, 21503, 267, 132, 256, 8461...","[-100, 2, 2, -100, -100, -100, -100, -100, 2, ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[name, 123670r.msg, =?koi8-r?b?68ht2nhoz9fbiod..."


In [28]:
CHUNCK_SIZE = 512

def lists_to_chuncks(row):
    n = CHUNCK_SIZE - 2
    new_row = row.copy()
    for key in ['attention_mask', 'input_ids', 'labels', 'token_type_ids']:
        FIRST = new_row[key][0]
        LAST = new_row[key][-1]
        new_row[key] = new_row[key][1:-1]
        new_row[key] = [[FIRST] + list(new_row[key][i:i + n]) + [LAST] for i in range(0, len(new_row[key]), n)]
    return new_row

In [29]:
train_tokenized = train_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
train_tokenized = train_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [30]:
test_tokenized = test_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
test_tokenized = test_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [31]:
train_tokenized.to_pickle('train_tokenized.pkl')
test_tokenized.to_pickle('test_tokenized.pkl')

In [32]:
tokenized_datasets = load_dataset('pandas', data_files={'train' : 'train_tokenized.pkl', 'test' : 'test_tokenized.pkl'})

Using custom data configuration default-cb1dc14db48b5297


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-cb1dc14db48b5297/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [33]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    1801.000000
mean      386.162132
std       158.276138
min         5.000000
25%       271.000000
50%       482.000000
75%       512.000000
max       512.000000
dtype: float64

In [34]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count    452.000000
mean     396.849558
std      143.963162
min        5.000000
25%      302.000000
50%      487.000000
75%      512.000000
max      512.000000
dtype: float64

Теперь все последовательности нужной длины

# Model

In [35]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Warning : модель надо обучить на наших данных

In [36]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [37]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [38]:
metric = load_metric("seqeval")

In [39]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [40]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.071414,0.592879,0.688230,0.637006,0.976219
2,No log,0.052717,0.728941,0.785265,0.756055,0.984867
3,0.070300,0.051787,0.761307,0.816712,0.788036,0.986970


TrainOutput(global_step=678, training_loss=0.05725368426612697, metrics={'train_runtime': 694.3969, 'train_samples_per_second': 0.976, 'total_flos': 770380182417600.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2051850240, 'init_mem_gpu_alloc_delta': 709342720, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 130506752, 'train_mem_gpu_alloc_delta': 2196961280, 'train_mem_cpu_peaked_delta': 284663808, 'train_mem_gpu_peaked_delta': 6504170496})

In [42]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9869702399501402,
 'eval_f1': 0.788036410923277,
 'eval_loss': 0.051787152886390686,
 'eval_mem_cpu_alloc_delta': 192512,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 390628864,
 'eval_precision': 0.7613065326633166,
 'eval_recall': 0.816711590296496,
 'eval_runtime': 18.2822,
 'eval_samples_per_second': 24.723}

In [43]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'GoodsString': {'f1': 0.788036410923277,
  'number': 1113,
  'precision': 0.7613065326633166,
  'recall': 0.816711590296496},
 'overall_accuracy': 0.9869702399501402,
 'overall_f1': 0.788036410923277,
 'overall_precision': 0.7613065326633166,
 'overall_recall': 0.816711590296496}

In [44]:
pd.DataFrame.from_dict(results).transpose().iloc[:-4, :-1]

,f1,number,precision
GoodsString,0.788036,1113.0,0.761307


In [45]:
pd.DataFrame.from_dict(results).transpose().iloc[-4:, 0].rename('result')

overall_precision    0.761307
overall_recall       0.816712
overall_f1           0.788036
overall_accuracy     0.986970
Name: result, dtype: float64

In [46]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 33.1MB/s 


In [47]:
from sklearn_crfsuite import metrics
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags, digits=3))

               precision    recall  f1-score   support

B-GoodsString      0.851     0.880     0.865      1109
I-GoodsString      0.870     0.868     0.869      1718
            O      0.994     0.994     0.994     48517

     accuracy                          0.987     51344
    macro avg      0.905     0.914     0.910     51344
 weighted avg      0.987     0.987     0.987     51344



In [48]:
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags[:-1], digits=3))

               precision    recall  f1-score   support

B-GoodsString      0.851     0.880     0.865      1109
I-GoodsString      0.870     0.868     0.869      1718

    micro avg      0.863     0.873     0.868      2827
    macro avg      0.861     0.874     0.867      2827
 weighted avg      0.863     0.873     0.868      2827

